<a href="https://colab.research.google.com/github/yashasangani/Machine-Learning/blob/main/Pipeline/titanic_without_using_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/yashasangani/Datasets/main/train.csv",delimiter = ',')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [7]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace = True)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


# Step 1: Train_test_split

In [8]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns = ["Survived"]),df["Survived"],test_size = 0.2,random_state = 0)
x_train.shape

(712, 7)

In [9]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2


# Step 2 : Applying Imputation

In [11]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy = "most_frequent")

x_train_age = si_age.fit_transform(x_train[["Age"]])
x_train_embarked = si_embarked.fit_transform(x_train[["Embarked"]])

x_test_age = si_age.transform(x_test[["Age"]])
x_test_embarked = si_embarked.transform(x_test[['Embarked']])

# Step 3 : One hot encoding Sex and Embarked columns

In [33]:
oe_sex = OneHotEncoder(sparse = False,handle_unknown = 'ignore')
oe_embarked =OneHotEncoder(sparse = False, handle_unknown = 'ignore' )

x_train_sex = oe_sex.fit_transform(x_train[["Sex"]])
x_train_embarked_new = oe_embarked.fit_transform(x_train_embarked)

x_test_sex = oe_sex.transform(x_test[["Sex"]])
x_test_embarked_new = oe_embarked.transform(x_test_embarked)

In [34]:
x_train.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,NaN,0,2,15.2458,C
439,2,male,31.0,0,0,10.5000,S
817,2,male,31.0,1,1,37.0042,C


In [35]:
x_train_rem = x_train.drop(columns = ["Sex","Age","Embarked"])
x_test_rem = x_test.drop(columns = ["Sex","Age","Embarked"])

In [36]:
x_train_transformed = np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked_new),axis = 1)

In [37]:
x_test_transformed = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked_new),axis = 1)

In [38]:
x_train_transformed.shape

(712, 10)

In [39]:
x_test_transformed.shape

(179, 10)

# Step 4 : Applying Decision tree classifier

In [40]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [41]:
y_pred = clf.predict(x_test_transformed)
y_pred

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1])

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.776536312849162

In [45]:
import pickle
pickle.dump(oe_sex,open('models/oe_sex.pkl','wb'))
pickle.dump(oe_embarked,open('models/oe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))